Instruções

- Façam grupos até 4 alunos  (não, não pode ser mais e não precisa ser o mesmo do outro trabalho).
- Escolham uma base de dados e apliquem  ao menos uma das técnicas vistas neste curso
- Postem nessa tarefa quem são os alunos componentes do grupo, ao menos uma amostra da base escolhida com os respectivos metadados.
- Pode ser uma base de outro trabalho de vocês ou uma base pública, desde que essa base não seja "óbvia" (tipo o Titanic). Outra observação, embora o grupo possa usar base pública, a análise deve ser feita pelo grupo com os códigos etc.. Se o trabalho apresentado for identificado como uma cópia de um trabalho publicado o grupo tira 0.
- Apresentação em power point de 15 a 30 minutos, explicando o intuito da análise, o desenvolvimento, os achados e conclusões. Também devo receber a base e o código usado para fazer a análise (vocês podem utilizar outra ferramenta diferente do python).

### Grupo
- Gustavo Schlieper Tessitore
- Lucas Lopes Amorim
- Isaac Higuchi


### Objetivo
Nesta competição, você irá prever as vendas para as milhares de famílias de produtos vendidos nas lojas Favorita localizadas no Equador.
 Os dados de treinamento incluem:

### Base de dados
- [Vendas no varejo](https://www.kaggle.com/competitions/store-sales-time-series-forecasting)
Colunas da base de dados:
- `store_nbr`: O número da loja onde o produto foi vendido
- `family`: A família de produtos a que o produto pertence
- `sales`: O valor de venda
- `onpromotion`: Quantidade de produtos vendidos em promoção dentro da família
- `date`: A data em que o produto foi vendido



In [ ]:
import pandas as pd

train_df = pd.read_parquet('store-sales-time-series-forecasting/train.parquet')
missing_values = train_df.isnull().sum()
descriptive_stats = train_df.describe()

missing_values, descriptive_stats

In [ ]:
# train test split
train_size = int(len(train_df) * 0.8)
train_df, test_df = train_df[:train_size], train_df[train_size:]

train_df.shape, test_df.shape

**Valores Faltantes**: 
Não há valores faltantes em nenhum dos campos do conjunto de dados.

**Estatísticas Descritivas**:
A coluna sales tem um valor mínimo de 0 e um valor máximo de 124,717. O valor médio das vendas é de aproximadamente 357.78, mas o desvio padrão é alto (1,101.998), indicando uma grande variação nas vendas.
A coluna onpromotion parece ser numérica, mas seus valores parecem indicar a contagem de produtos em promoção em vez de um valor binário (0 ou 1). O valor máximo é 741, o que pode representar a contagem de produtos em promoção para uma determinada família de produtos em uma determinada loja em um determinado dia.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Setting up the aesthetics for the plots
sns.set_style("whitegrid")

# Initialize the figure
plt.figure(figsize=(20, 15))

# 1. Distribution of Sales
plt.subplot(2, 2, 1)
sns.histplot(train_df['sales'], bins=50, kde=True)
plt.title('Distribution of Sales')
plt.xlabel('Sales')
plt.ylabel('Frequency')

# 2. Average Sales by Product Family
plt.subplot(2, 2, 2)
avg_sales_family = train_df.groupby('family')['sales'].mean().sort_values(ascending=False)
sns.barplot(x=avg_sales_family.index, y=avg_sales_family.values, palette='viridis')
plt.title('Average Sales by Product Family')
plt.xlabel('Product Family')
plt.ylabel('Average Sales')
plt.xticks(rotation=90)

# 3. Average Sales by Store
plt.subplot(2, 2, 3)
avg_sales_store = train_df.groupby('store_nbr')['sales'].mean().sort_values(ascending=False)
sns.barplot(x=avg_sales_store.index, y=avg_sales_store.values, palette='magma')
plt.title('Average Sales by Store')
plt.xlabel('Store Number')
plt.ylabel('Average Sales')

# 4. Sales Trend over Time
plt.subplot(2, 2, 4)
train_df['date'] = pd.to_datetime(train_df['date'])
sales_trend = train_df.groupby('date')['sales'].sum()
sns.lineplot(x=sales_trend.index, y=sales_trend.values, color='blue')
plt.title('Sales Trend over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')

# Adjusting the layout
plt.tight_layout()
plt.show()

**Distribuição das Vendas**: A maioria das vendas está concentrada em valores baixos, com alguns valores extremamente altos, indicando a presença de outliers ou dias/eventos de vendas excepcionais.
**Vendas Médias por Família de Produtos**: Algumas famílias de produtos têm vendas médias significativamente mais altas do que outras. Isso pode ser devido à natureza do produto, à demanda do cliente ou a promoções.
**Vendas Médias por Loja**: A distribuição das vendas médias varia entre as lojas. Algumas lojas têm vendas médias significativamente mais altas, o que pode ser devido à sua localização, tamanho ou clientela.
**Tendência de Vendas ao Longo do Tempo**: Há uma tendência ascendente nas vendas ao longo do tempo, com algumas flutuações visíveis. Também há picos sazonais visíveis, possivelmente indicando eventos ou feriados que impulsionam as vendas.


In [ ]:
from pmdarima import auto_arima


# Aggregating the sales data by date for time series forecasting
time_series_data = train_df.groupby('date')['sales'].sum().reset_index()
time_series_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

In [ ]:
best_arima_model_stepwise = auto_arima(time_series_data['y'], 
                              seasonal=True, 
                              m=12, # Assuming monthly seasonality
                              trace=True, 
                              error_action='ignore', 
                              suppress_warnings=True, 
                              stepwise=True)

# Displaying the best ARIMA order
best_arima_model_stepwise.summary()

In [ ]:
best_arima_model= auto_arima(time_series_data['y'], 
                              seasonal=True, 
                              m=12, # Assuming monthly seasonality
                              trace=True, 
                              error_action='ignore', 
                              suppress_warnings=True, 
                              stepwise=False)

# Displaying the best ARIMA order
best_arima_model.summary()

**ARIMA(5,1,0)**:
Menos complexo, o que pode ser benéfico para evitar o sobreajuste.
O AIC é 44204.715, que é relativamente mais alto em comparação com o outro modelo.
O teste Ljung-Box indica autocorrelações nos resíduos, o que não é ideal.

**ARIMA(5,1,5) (resultado do stepwise)**:
Modelo mais complexo com 5 termos autoregressivos e 5 termos de média móvel.
O AIC é 43560.904, que é mais baixo do que o ARIMA(5,1,0), indicando um melhor ajuste aos dados.
O teste Ljung-Box indica que os resíduos são essencialmente aleatórios, o que é uma boa característica.
O coeficiente de determinação é mais alto, indicando uma melhor adequação aos dados.

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

order_map = {
    "stepwise": (5, 1, 5),
    "non-stepwise": (5, 1, 0)
}

# Fit the SARIMAX model with the identified order
model = SARIMAX(time_series_data['y'], order=order_map.get('stepwise'))
results = model.fit(disp=-1)

# Display the model summary
results.summary()

In [ ]:
# Extracting the dates from the test dataset
forecast_dates = test_df['date'].unique()

# Using the ARIMA(5,1,5) model to forecast
forecast = results.forecast(steps=len(forecast_dates))

# Creating a DataFrame for the forecasted values
forecast_df = pd.DataFrame({'date': forecast_dates, 'predicted_sales': forecast})

forecast_df.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Aggregating the test data to get the actual sales for the forecasted dates
actual_sales = test_df.groupby('date')['sales'].sum()

# Calculating the validation metrics
mae = mean_absolute_error(actual_sales, forecast)
mse = mean_squared_error(actual_sales, forecast)
rmse = np.sqrt(mse)

mae, mse, rmse

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    """Calculate MAPE given true and predicted values"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(actual_sales, forecast)
mape